Module 20 Team Proj, food_deserts, database setup

In [2]:
# Installing Spark

import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,376 kB]
Get:12 https://cloud.r-project.org/bi

In [3]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import datetime as dt

In [4]:
# Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-11 20:10:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-11 20:10:14 (6.64 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
# Start a Spark session with an additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Read in data from S3 Bucket
from pyspark import SparkFiles

url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_access_research_atlas.csv"
spark.sparkContext.addFile(url)
food_access_research_atlas_df = spark.read.csv(SparkFiles.get("food_access_research_atlas.csv"), sep=",", header=True, inferSchema=True)

In [7]:
food_access_research_atlas_df.show(5)

+-----------+-------+-------+-----+-------+-------+-----------------+--------+---------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------+-----------+---------------+-----------+---------------+-------------+------------------+-----------+----------------+-----------+----------------+-----------+----------------+-----------+----------------+-----------+---------------+-------------+------------------+-----------+---------------+-----------+---------------+-----------+---------------+-----------+-----------+-----------+------------+-----------+------------+-----------+---------------+-----------+-------------+-----------+-------------+-----------+-------------+----------+-------------+-----------+------------+------

In [8]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/food_desert_austin_censustract.csv"
spark.sparkContext.addFile(url)
fooddesert_austin_censustract_df = spark.read.csv(SparkFiles.get("food_desert_austin_censustract.csv"), sep=",", header=True, inferSchema=True)

In [9]:
fooddesert_austin_censustract_df.show(5)

+-----+-----------+-----+-------+-----+-------+-------+-----------------+--------+-------------------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+------------------+------------------+-----------------+-----------+-----------+-----------+------------------+------------------+------------------+---------------+-----------+-------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+--------------------+-----------+--------------------+-----------------+--------------------+------------------+-------------------+-----------+-------------------+------------------+--------------------+------------------+---------------+-----------------+------------------+------------------+-------------------+------------------+----

In [10]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/census_tract_shapefiles_all.csv"
spark.sparkContext.addFile(url)
census_tract_shapefiles_all_df = spark.read.csv(SparkFiles.get("census_tract_shapefiles_all.csv"), sep=",", header=True, inferSchema=True)

In [11]:
# Rename column GEOID to CensusTract
census_tract_shapefiles_all_df = census_tract_shapefiles_all_df.withColumnRenamed("GEOID", "CensusTract")

In [12]:
census_tract_shapefiles_all_df.show(5)

+-----------+----------+------------+-------------+--------------+
|CensusTract|PointOrder|SubPolygonId|PointLatitude|PointLongitude|
+-----------+----------+------------+-------------+--------------+
| 1001020100|         1|           1|      32.4988|       -86.482|
| 1001020100|         2|           1|      32.4915|      -86.4787|
| 1001020100|         3|           1|      32.4764|      -86.4812|
| 1001020100|         4|           1|       32.466|      -86.4751|
| 1001020100|         5|           1|      32.4597|      -86.4753|
+-----------+----------+------------+-------------+--------------+
only showing top 5 rows



In [13]:
url ="https://dataviz20-bucket.s3.us-east-2.amazonaws.com/Strategic_Measure_HE.F.2__Number_and_percentage_of_food_insecure_residents_that_are_living_in_proximity_to_a_City-supported_fresh_food_access_point.csv"
spark.sparkContext.addFile(url)
percent_foodinsecure_df = spark.read.csv(SparkFiles.get("Strategic_Measure_HE.F.2__Number_and_percentage_of_food_insecure_residents_that_are_living_in_proximity_to_a_City-supported_fresh_food_access_point.csv"), sep=",", header=True, inferSchema=True)

In [14]:
percent_foodinsecure_df.show(5)

+--------------------+------------------+-----------------------+------------------------+------------------------+
|                Date|      Census Tract|Census Tract Population|Food Insecure Population|Food Insecure Percentage|
+--------------------+------------------+-----------------------+------------------------+------------------------+
|01/01/2020 12:00:...|   Census Tract 10|                   3777|                     780|                   0.206|
|01/01/2020 12:00:...|   Census Tract 11|                   6608|                    1230|                   0.186|
|01/01/2020 12:00:...|   Census Tract 12|                   5513|                     970|                   0.177|
|01/01/2020 12:00:...|Census Tract 13.05|                   5876|                    1210|                   0.207|
|01/01/2020 12:00:...|Census Tract 13.07|                   3747|                     710|                   0.191|
+--------------------+------------------+-----------------------+-------

In [16]:
# Join two of the DataFrames
fooddesert_austin_censusshapes_df = fooddesert_austin_censustract_df.join(census_tract_shapefiles_all_df, on="CensusTract", how="inner")
fooddesert_austin_censusshapes_df.show(5)

+-----------+-----+-----+-------+-----+-------+-------+-----------------+--------+--------+-----------------+--------------------+-----------------+------------------+--------+---------------+-----------+------------------+--------+-----------+--------+-------------+---------+----------+----------+------------------+------------------+------------------+---------+-----------+-----------+----------+-----------------+--------------+------------------+---------------+-----------+-------------------+------------------+------------------+------------------+----------------+------------------+----------------+-----------+--------------------+-----------+--------------------+-----------------+---------------+------------------+------------------+-----------+---------------+-----------+---------------+-----------+---------------+-----------+-----------+---------+-------------------+-----------+------------+-----------+---------------+-----------+------------------+-----------+-------------+---

PostgreSQL on AWS setup

In [17]:
# Configure settings for AWS RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c5qcqhh5xq62.us-east-2.rds.amazonaws.com:5432/Module20_food_deserts"
config = {"user":"postgres", 
          "password": "Data1UT$", 
          "driver":"org.postgresql.Driver"}

In [18]:
# Write  _df to table in AWS RDS
fooddesert_austin_censusshapes_df.write.jdbc(url=jdbc_url, table='fooddesert_austin_censusshapes', mode=mode, properties=config)

In [19]:
# Write  _df to table in AWS RDS
food_access_research_atlas_df.write.jdbc(url=jdbc_url, table='food_access_research_atlas', mode=mode, properties=config)

In [20]:
# Write  _df to table in AWS RDS
fooddesert_austin_censustract_df.write.jdbc(url=jdbc_url, table='fooddesert_austin_censustract', mode=mode, properties=config)

In [21]:
# Write  _df to table in AWS RDS
percent_foodinsecure_df.write.jdbc(url=jdbc_url, table='percent_foodinsecure', mode=mode, properties=config)